In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

In [2]:
# Labels for the different types of images the CNN will predict
class_labels = {
    0: 'Agony',
    1: 'Happy',
    2: 'Neutral',
    3: 'Sad',
    4: 'Scared'
}

In [3]:
# Define the path to your data and model
base_path = r"C:\Projects\Images"
model_save_path = os.path.join(base_path, 'Models', 'last_best_model.h5')

In [4]:
# Load the model without compiling it
model = load_model(model_save_path, compile=False)

# Compile the model manually
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

print("Loaded model:", model_save_path)

Loaded model: C:\Projects\Images\Models\last_best_model.h5


In [5]:
# Callbacks
model_checkpoint = ModelCheckpoint(
    model_save_path,  # Path where the model will be saved
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,  # Only save the model if val_accuracy has improved
    mode='max',  # Save the model when val_accuracy is at its max
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    min_delta=0.001,  # Minimum change to qualify as an improvement
    patience=20,  # How many epochs to wait before stopping
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored quantity
    mode='min',  # Stop training when the quantity monitored has stopped decreasing
    verbose=1
)

In [6]:
# Prepare image for prediction using saved model
def prepare_image(file_path):
    img = image.load_img(file_path, target_size=(240, 240))  # Resize image
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Scale image
    return img_array

def LoadSavedModel(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Model file not found: {file_path}")
    # Load the model without compiling
    model = load_model(file_path, compile=False)
    print("Loaded model: ", file_path)
    return model

In [7]:
# Build Test and Train using a CNN Model
def BuildTrainAndTestModel():
    # Set Seed value for consistent initialization
    seed_value = 100

    # Set the PYTHON HASHSEED environment variable at a fixed value
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # Set the python built-in pseudo-random generator at a fixed value
    random.seed(seed_value)

    # Set the numpy pseudo-random generator at a fixed value
    np.random.seed(seed_value)

    # Set the tensorflow pseudo-random generator at a fixed value
    tf.random.set_seed(seed_value)

    # Create Image Data Generators
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Create the Models directory if it doesn't exist
    models_path = os.path.join(base_path, 'Models')
    if not os.path.exists(models_path):
        os.makedirs(models_path)

    # Training Data
    train_generator = train_datagen.flow_from_directory(
        os.path.join(base_path, 'Train'),  # Path to training data
        target_size=(240, 240),  # Resize images to 240x240
        batch_size=32,
        class_mode='categorical'  # Since we're classifying multiple classes
    )

    # Validation data
    validation_generator = val_datagen.flow_from_directory(
        os.path.join(base_path, 'Validation'),
        target_size=(240, 240),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(base_path, 'Test'),
        target_size=(240, 240),
        batch_size=32,
        class_mode='categorical'
    )

    # Create the CNN Model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 3)),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(5, activation='softmax')  # 5 classes
    ])

    # Model compilation using optimizer Adam
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    model_checkpoint = ModelCheckpoint(
        os.path.join(base_path, 'Models', 'last_best_model.h5'),  # Path where the model will be saved
        monitor='val_accuracy',  # Monitor validation accuracy
        save_best_only=True,  # Only save the model if val_accuracy has improved
        mode='max',  # Save the model when val_accuracy is at its max
        verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',  # Monitor the validation loss
        min_delta=0.001,  # Minimum change to qualify as an improvement
        patience=20,  # How many epochs to wait before stopping
        restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored quantity
        mode='min',  # Stop training when the quantity monitored has stopped decreasing
        verbose=1
    )

    # Train the Model
    train = model.fit(
        train_generator,
        steps_per_epoch=100,  # Number of batches per epoch
        epochs=50,
        validation_data=validation_generator,
        validation_steps=50,  # Number of validation batches
        callbacks=[model_checkpoint, early_stopping]  # Early stopping and saving
    )

    # Verify the model save path
    if not os.path.exists(os.path.join(base_path, 'Models', 'last_best_model.h5')):
        print("Model was not saved correctly.")
    else:
        print("Model saved successfully.")

    # Evaluate the Model
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"Test accuracy: {test_acc}")

In [8]:
def PredictImage(model, file_path):
    # Prepare image
    img_to_predict = prepare_image(file_path)

    # Predict
    predictions = model.predict(img_to_predict)

    # Get the predicted class for the image based on the highest probability
    predicted_class = np.argmax(predictions, axis=1)

    # Get the name of the predicted class for the image
    predicted_class_name = class_labels[predicted_class[0]]
    print('The predicted facial expression is: ', predicted_class_name)

# Ensure the model exists before attempting to load it
if not os.path.exists(model_save_path):
    print(f"Model file does not exist: {model_save_path}")
else:
    # Load the saved model for the CNN without compiling
    model = LoadSavedModel(model_save_path)

    # Compile the model manually with the Adam optimizer
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Predict the image class from files stored in the Pics folder
    PredictImage(model, r'C:\Projects\Images\988.jpg')  # Test image for Agony

Loaded model:  C:\Projects\Images\Models\last_best_model.h5
The predicted facial expression is:  Agony
